In [30]:
import numpy as np
from tqdm import tqdm
import json
import time
import datetime
from ldpc import BpOsdDecoder
from bposd.css import css_code
import scipy
from lifted_hgp_4d import *
from mm_qc_pega.graphs import QC_tanner_graph
import numpy as np
import ldpc.protograph as pt
from distance import *
from css_ss_decode_sim import *
import scipy.sparse as sp

def write_alist_mod2sparse(H, path):
    print(path)
    H = sp.csr_matrix(H)
    Hc = H.tocsc()

    M, N      = H.shape
    col_w     = Hc.getnnz(axis=0)
    row_w     = H .getnnz(axis=1)
    max_col_w = int(col_w.max())
    max_row_w = int(row_w.max())

    nlist = [list(Hc.indices[Hc.indptr[j]:Hc.indptr[j+1]] + 1) for j in range(N)]
    mlist = [list(H.indices[H.indptr[i]:H.indptr[i+1]] + 1) for i in range(M)]

    with open(path, "w") as f:
        f.write(f"{N} {M}\n")
        f.write(f"{max_col_w} {max_row_w}\n")
        f.write(" ".join(map(str, col_w)) + "\n")
        f.write(" ".join(map(str, row_w)) + "\n")

        for col in nlist:
            f.write(" ".join(map(str, col)) + "\n")  # No padding

        for row in mlist:
            f.write(" ".join(map(str, row)) + "\n")  # No padding

def has_abandon_risk_ratio(H, min_ratio=0.75, max_ratio=1):
    """
    Check if H matrix is proportionally close between variables (N) and checks (M).
    Warn if not.
    
    Parameters:
        H : scipy.sparse matrix (or ndarray)
        min_ratio : float
        max_ratio : float
            Acceptable bounds for (N / M)
    """
    M, N = H.shape
    ratio = M / N

    print(f"Matrix size: M={M} rows x N={N} columns")
    print(f"N / M ratio: {ratio:.3f}")

    return (min_ratio <= ratio <= max_ratio)

In [3]:
n1 = 2
n2 = 7
L = n1*n2
a = pt.array([[(0,1)]])
d = pt.array([[(0,n2)]])  
qcode = bias_tailored_lhp_4d(5, a=a, b=a, c=a, d=d)

qcode.test()
write_alist_mod2sparse(qcode.hx, f"{path}4D_biased_aaad_{n1}_{n2}_hx.alist")
write_alist_mod2sparse(qcode.mx,  f"{path}4D_biased_aaad_{n1}_{n2}_mx.alist")
write_alist_mod2sparse(qcode.lx,  f"{path}4D_biased_aaad_{n1}_{n2}_lx.alist")

hx bin shape (20, 30)
hz bin shape (20, 30)
(hz @ hx.T) + (hx @ hz.T) == 0?: True
(mx @ hx @ hz.T @ mz.T) == 0?: True


In [4]:
qcode.test()
write_alist_mod2sparse(qcode.hx, f"{path}4D_biased_aaad_{n1}_{n2}_hx.alist")
write_alist_mod2sparse(qcode.mx,  f"{path}4D_biased_aaad_{n1}_{n2}_mx.alist")
write_alist_mod2sparse(qcode.lx,  f"{path}4D_biased_aaad_{n1}_{n2}_lx.alist")

<Unamed stabiliser code>, [[30,14,nan]]
 -Block dimensions: Pass
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
 -PCMs commute hz@hx.T==0: Pass
	-lx \in ker{hz} AND lz \in ker{hx}: Pass
<class 'scipy.sparse._csr.csr_matrix'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

True

In [43]:
import numpy as np

def save_alist(name, mat, j=None, k=None):

    H=np.copy(mat)
    H=H.T

    '''
    Function converts parity check matrix into the format required for the RN decoder
    '''

    if j is None:
        j=int(max(H.sum(axis=0)))


    if k is None:
        k=int(max(H.sum(axis=1)))


    m, n = H.shape # rows, cols
    f = open(name, 'w')
    print(n, m, file=f)
    print(j, k, file=f)

    for col in range(n):
        print( int(H[:, col].sum()), end=" ", file=f)
    print(file=f)
    for row in range(m):
        print( int(H[row, :].sum()), end=" ", file=f)
    print(file=f)

    for col in range(n):
        for row in range(m):
            if H[row, col]:
                print( row+1, end=" ", file=f)
        print(file=f)

    for row in range(m):
        for col in range(n):
            if H[row, col]:
                print(col+1, end=" ", file=f)
        print(file=f)
    f.close()

In [47]:
type(qcode.hx.toarray())

numpy.ndarray

In [54]:
path = 'single_shot_3D_HGP/codes/4d/NEW_4D_biased_aaad_2_7'
save_alist(f"{path}_hx.alist", qcode.hx.toarray())
save_alist(f"{path}_mx.alist", qcode.mx.toarray())
save_alist(f"{path}_lx.alist", qcode.lx.toarray())

In [60]:
compute_min_hamming_distance(qcode.mx.toarray())

Started computation at 2025-04-28 16:11:51.752233
Generator matrix has 14 rows and 56 columns.
Finding Hamming distance...
Minimum non-zero Hamming distance found: 8
Contributing rows from G: [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13]
XOR Result (codeword): [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
Ended computation at 2025-04-28 16:11:51.788093
Total computation time: 0:00:00.035860


8

In [ ]:
# import numpy as np
# from alist import save_alist

# hamming_matrix=np.array([[1,0,0,1,1,0,1],
#                          [0,1,0,1,0,1,1],
#                          [0,0,1,1,1,1,0]])

# save_alist("hamming_d_3.alist",hamming_matrix)

In [34]:
path = 'single_shot_3D_HGP/codes/4d/'

n1 = 2
n2 = 7
L = n1*n2
a = pt.array([[(0,1)]])
d = pt.array([[(0,n2)]])     # twist in δ_D
qcode = bias_tailored_lhp_4d(L, a=a, b=a, c=a, d=d)

# if not has_abandon_risk_ratio(qcode.hx):
write_alist_mod2sparse(qcode.hx, f"{path}4D_biased_aaad_{n1}_{n2}_hx.alist")
write_alist_mod2sparse(qcode.mx,  f"{path}4D_biased_aaad_{n1}_{n2}_mx.alist")
write_alist_mod2sparse(qcode.lx,  f"{path}4D_biased_aaad_{n1}_{n2}_lx.alist")

hx bin shape (56, 84)
hz bin shape (56, 84)
(hz @ hx.T) + (hx @ hz.T) == 0?: True
(mx @ hx @ hz.T @ mz.T) == 0?: True
single_shot_3D_HGP/codes/4d/4D_biased_aaad_2_7_hx.alist
single_shot_3D_HGP/codes/4d/4D_biased_aaad_2_7_mx.alist
single_shot_3D_HGP/codes/4d/4D_biased_aaad_2_7_lx.alist


In [25]:
f"{path}_4D_biased_aaad_{n1}_{n2}_hx.alist"

'single_shot_3D_HGP/codes/4d/_4D_biased_aaad_2_7_hx.alist'

In [22]:
qcode.N

84

In [181]:
a = pt.array([[(0,1)]])


In [199]:
A = pt.array([
    [(0,1)],
    [(1,1)],
    [(2,1)]
])


In [201]:
print(A)

[[λ(0,1)]
 [λ()]
 [λ(1,2)]]


In [241]:
n1, n2 = 2, 7
L = n1 * n2

# one row per added stabiliser --------------------------
row1 = [[(0, 1)]]          # original face
row2 = [[(1, 0)]]          # second independent face
row3 = [[(1, 1)]]          # third face  (diagonal)

A = pt.array([*row1, *row2, *row3])   # shape 3 × 1
B = A.copy                        # keep identical shapes
C = A.copy
# D must **also** have 3 rows, even if twist edge repeats
D = pt.array([[(0, n2)], [(0, n2)], [(0, n2)]])  # 3 × 1

In [243]:
A

array([[(0,1)],
       [(0,1)],
       [()]], dtype=object)

In [7]:
# choose R (rows) >= C (cols) so M ≥ N
R, C = 3, 1
twist = (0, n2)               # keeps the bias-tailored twist

row_X1 = [(0, 1), twist]      # original face  +  twist edge
row_X2 = [(1, 0), twist]      # orthogonal face + twist
row_X3 = [(1, 1), twist]      # diagonal face  + twist

A = pt.array([row_X1, row_X2, row_X3])   # 3 × 1
D = pt.array([[twist]] * R)              # 3 × 1


In [11]:
# choose R (rows) >= C (cols) so M ≥ N
R, C = 3, 1
twist = (0, n2)               # keeps the bias-tailored twist

row_X1 = [(0, 1), twist]      # original face  +  twist edge
row_X2 = [(1, 0), twist]      # orthogonal face + twist
row_X3 = [(1, 1), twist]      # diagonal face  + twist

A = pt.array([row_X1, row_X2, row_X3])   # 3 × 1
D = pt.array([[twist]] * R)              # 3 × 1


In [15]:
A.shape

(3, 2)

In [17]:
D.shape

(3, 1)

In [13]:
hx_p = pt.hstack(construct_HX(A,A,A,D))   # proto only
hz_p = pt.hstack(construct_HZ(A,A,A,D))
assert hx_p.shape[1] == hz_p.shape[1], "column mismatch"

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 2 and the array at index 1 has size 1

In [9]:
hx_p = pt.hstack(construct_HX(A,A,A,D))   # proto only
hz_p = pt.hstack(construct_HZ(A,A,A,D))
assert hx_p.shape[1] == hz_p.shape[1], "column mismatch"


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 2 and the array at index 1 has size 1

In [4]:
import ldpc.protograph as pt
from lifted_hgp_4d import lifted_hgp_4d               # or bias_tailored_lifted_product
                                                         # depending on which builder you use

# ------------------------------------------------------------
# 1)  Choose basic parameters
# ------------------------------------------------------------
n1, n2        = 2, 7                    # horizontal / vertical cell count
L             = n1 * n2                 # total 4-cells
R, C          = 3, 1                    # ***rows > cols  ⇒  M >= N  after lifting***

# ------------------------------------------------------------
# 2)  Define edge lists for each protograph cell
#     (one list per *row*, identical across A,B,C for simplicity)
# ------------------------------------------------------------
face1 = [(0, 1)]                        # original X-face
face2 = [(1, 0)]                        # orthogonal face
face3 = [(1, 1)]                        # diagonal face   (extra stabiliser)

row_edges = [face1, face2, face3]       # 3 independent rows

# Build a 3 × 1 protograph from those rows
A = pt.array(row_edges)                 # shape (3, 1)

# ------------------------------------------------------------
# 3)  Twist protograph D  –– must have same shape (3 × 1)
# ------------------------------------------------------------
twist_edge = (0, n2)                    # shift that forces the “delta_D” twist
D = pt.array([ [twist_edge] ] * R)      # replicate so D has 3 rows

# ------------------------------------------------------------
# 4)  Construct the lifted code
# ------------------------------------------------------------
qcode = lifted_hgp_4d(L, a=A, b=B, c=C, d=D)   # or bias_tailored_lifted_product 
print("HX shape:", qcode.hx.shape)
print("HZ shape:", qcode.hz.shape)


hx bin shape (168, 84)
hx bin shape (56, 252)


Exception: Error: hx and hz matrices must have equal numbers of columns!

In [229]:
# import ldpc.protograph as pt
# from lifted_hgp_4d import lifted_hgp_4d

# n1, n2 = 2, 7
# L = n1 * n2

# # --- richer protographs -------------------------------------------------
# a = pt.array([[(0, 1), (1, 0), (1, 1)]])   # 3 edges in same cell
# b = a                                      # keep identical for simplicity
# c = a
# d = pt.array([[(0, n2)]])                  # twist unchanged

# # --- build code ----------------------------------------------------------
# qcode = lifted_hgp_4d(L, a=a, b=b, c=c, d=d)
# print("HX shape:", qcode.hx.shape)         # should be 3L × 3L


Matrix size: M=56 rows x N=84 columns
N / M ratio: 0.667


False